## Iteration with purrr

よく使うループ処理はそれ自体が関数となっている. 
関数プログラミングを用いることで, ループ処理を隠蔽しよう. 


In [1]:
library (tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


### For Loop

普通のループは次のような処理となる. 

In [9]:
# set data
df <- lapply(1:4, rnorm, n=50) %>%
      set_names(letters[1:4]) %>%
      as_tibble()

# calc
output <- vector("double", ncol(df))
for (ii in seq_along(df)) {
    output[[ii]] <- median (df[[ii]])
}
output

[1] 0.8704937 2.0265226 2.9486315 3.6673980

### purrr
##### shortcut

In [13]:
# 無名関数はチルダ等を駆使して書き換えることが可能である
models <- 
    mtcars %>%
    split(.$cyl) %>%
    map(~lm (mpg ~ wt, data=.))
models

$`4`

Call:
lm(formula = mpg ~ wt, data = .)

Coefficients:
(Intercept)           wt  
     39.571       -5.647  


$`6`

Call:
lm(formula = mpg ~ wt, data = .)

Coefficients:
(Intercept)           wt  
      28.41        -2.78  


$`8`

Call:
lm(formula = mpg ~ wt, data = .)

Coefficients:
(Intercept)           wt  
     23.868       -2.192  



In [14]:
# 属性名を与えることで, 属性を取り出すことも可能のである
models %>%
    map(summary) %>%
    map("r.squared")

$`4`
[1] 0.5086326

$`6`
[1] 0.4645102

$`8`
[1] 0.4229655

In [26]:
# 要素位置を与えることでも抽出が可能である
x <- list(list (1:3), list (4:6), list (7:9))
x %>% map( ~ map(., 2))

[[1]]
[[1]][[1]]
[1] 2


[[2]]
[[2]][[1]]
[1] 5


[[3]]
[[3]][[1]]
[1] 8

### Dealing with Failure

大量のループ処理が, 一カ所の失敗でなにも返されないのは衝撃的むかつく. 
そこで`safely()`関数を利用して関数を修正することを提案する.　
safelyは次の二つを返す. 

- result: 演算結果. エラーがあった場合にはそこをNULLとして返す
- error: エラーオブジェクト. resultの反対で成功している場合にはNULLとなる




In [31]:
x <- -2:2
safe_log <- safely(log)
y <- safe_log(x) # 警告が出るが,　yには演算結果が取得できている. 
print (y)

Warning message in .f(...):
" 計算結果が NaN になりました "

$result
[1]       NaN       NaN      -Inf 0.0000000 0.6931472

$error
NULL



In [34]:
# possibillyはsafelyのシンプル版で, quietlyはエラーを削除する
x %>% map(possibly(log, NA_real_))
x %>% map(quietly(log))

Warning message in .f(...):
" 計算結果が NaN になりました "Warning message in .f(...):
" 計算結果が NaN になりました "

[[1]]
[1] NaN

[[2]]
[1] NaN

[[3]]
[1] -Inf

[[4]]
[1] 0

[[5]]
[1] 0.6931472

[[1]]
[[1]]$result
[1] NaN

[[1]]$output
[1] ""

[[1]]$warnings
[1] " 計算結果が NaN になりました "

[[1]]$messages
character(0)


[[2]]
[[2]]$result
[1] NaN

[[2]]$output
[1] ""

[[2]]$warnings
[1] " 計算結果が NaN になりました "

[[2]]$messages
character(0)


[[3]]
[[3]]$result
[1] -Inf

[[3]]$output
[1] ""

[[3]]$warnings
character(0)

[[3]]$messages
character(0)


[[4]]
[[4]]$result
[1] 0

[[4]]$output
[1] ""

[[4]]$warnings
character(0)

[[4]]$messages
character(0)


[[5]]
[[5]]$result
[1] 0.6931472

[[5]]$output
[1] ""

[[5]]$warnings
character(0)

[[5]]$messages
character(0)

###  Mapping over Multiple Arguments

In [38]:
Filter(is.numeric, iris) %>% pmap(max) %>% flatten_dbl

[1] 5.1 4.9 4.7 4.6 5.0 5.4 4.6 5.0 4.4 4.9 5.4 4.8 4.8 4.3 5.8 5.7 5.4 5.1
 [19] 5.7 5.1 5.4 5.1 4.6 5.1 4.8 5.0 5.0 5.2 5.2 4.7 4.8 5.4 5.2 5.5 4.9 5.0
 [37] 5.5 4.9 4.4 5.1 5.0 4.5 4.4 5.0 5.1 4.8 5.1 4.6 5.3 5.0 7.0 6.4 6.9 5.5
 [55] 6.5 5.7 6.3 4.9 6.6 5.2 5.0 5.9 6.0 6.1 5.6 6.7 5.6 5.8 6.2 5.6 5.9 6.1
 [73] 6.3 6.1 6.4 6.6 6.8 6.7 6.0 5.7 5.5 5.5 5.8 6.0 5.4 6.0 6.7 6.3 5.6 5.5
 [91] 5.5 6.1 5.8 5.0 5.6 5.7 5.7 6.2 5.1 5.7 6.3 5.8 7.1 6.3 6.5 7.6 4.9 7.3
[109] 6.7 7.2 6.5 6.4 6.8 5.7 5.8 6.4 6.5 7.7 7.7 6.0 6.9 5.6 7.7 6.3 6.7 7.2
[127] 6.2 6.1 6.4 7.2 7.4 7.9 6.4 6.3 6.1 7.7 6.3 6.4 6.0 6.9 6.7 6.9 5.8 6.8
[145] 6.7 6.7 6.3 6.5 6.2 5.9

### Invoking Different Functions

In [39]:
f <-  c("runif", "rnorm", "rpois")
param <- list (
    list (min = -1, max = 1), 
    list (sd = 5), 
    list (lambda = 10)
)
invoke_map(f, param, n = 5) %>% str

List of 3
 $ : num [1:5] 0.7991 0.8913 -0.6126 -0.0797 -0.061
 $ : num [1:5] 2.23 -1.87 -3.57 -3.6 3.79
 $ : int [1:5] 9 8 7 8 12


In [42]:
sim <- tribble (
    ~f, ~params, 
    "runif", list(min=-1,max=1), 
    "rnorm", list(sd=5), 
    "rpois", list(lambda=10)
)
sim %>%
    mutate (sim = invoke_map(f, params, n = 10)) %>%
    print()

# A tibble: 3 x 3
      f     params        sim
  <chr>     <list>     <list>
1 runif <list [2]> <dbl [10]>
2 rnorm <list [1]> <dbl [10]>
3 rpois <list [1]> <int [10]>


### Walk

副作用がないmapです. プロットを保存する際などに便利です. 

### Other Paterns of For Loop

In [47]:
iris %>% 
    keep(is.factor) %>%
    head()

Species
setosa
setosa
setosa
setosa
setosa
setosa


In [48]:
iris %>%
    discard(is.factor) %>%
    head()

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
5.1,3.5,1.4,0.2
4.9,3.0,1.4,0.2
4.7,3.2,1.3,0.2
4.6,3.1,1.5,0.2
5.0,3.6,1.4,0.2
5.4,3.9,1.7,0.4


In [49]:
iris %>%
    some(is.factor)

[1] TRUE

In [50]:
iris %>%
    every(is.numeric)

[1] FALSE

In [53]:
# detectは条件を満たす最初の値, 
# detect_indexはその要素位置
iris %>% detect( ~ is.factor(.)) %>% head()
iris %>% detect_index( ~ is.factor(.))

[1] setosa setosa setosa setosa setosa setosa
Levels: setosa versicolor virginica

[1] 5